In [22]:
exemple = ["dog", "deer", "deal", "death", "dean", "dear", "dare"]
exemple.sort()

def get_matches(word):
    matches = []
    for w in exemple:
        if w.startswith(word):
            matches.append(w)
    return matches

In [23]:
import tkinter as tk

class AutocompleteText(tk.Text):
    def __init__(self, *args, **kwargs):
        self.callback = kwargs.pop("autocomplete", None)
        super().__init__(*args, **kwargs)
        self.bind("<Any-KeyRelease>", self._autocomplete)
        self.bind("<Tab>", self._handle_tab)
        self.bind("<Button-1>", self.click)

    def _handle_tab(self, event):
        tag_ranges= self.tag_ranges("autocomplete")
        if tag_ranges:
            self.mark_set("insert", tag_ranges[1])
            self.tag_remove("sel", "1.0", "end")
            self.tag_remove("autocomplete", "1.0", "end")

            return "break"

    def _autocomplete(self, event):
        if event.char and self.callback:
            word = self.get("insert-1c wordstart", "insert-1c wordend")

            matches = self.callback(word)

            if matches:
                remainder = matches[0][len(word):]
                insert = self.index("insert")
                self.insert(insert, remainder, ("sel", "autocomplete"))
                self.mark_set("insert", insert)
            
            texto = "Outras Sugestões: \n"
            for match in matches[1:]:
                texto += match + "\n"
            texto_outras_opcoes["text"] = texto
            

    # quando clicar na dentro da caixa de texto, print da linha e coluna
    def click(self, event):
        linha = int(self.index(tk.INSERT).split(".")[0])
        linha += 1
        linha = str(linha)
        #pegar o texto da linha
        texto = self.get(linha + ".0", linha + ".end")
        print(texto)
          


root = tk.Tk()

# titulo
root.title("Autocomplete")

# Fullscreen
root.attributes("-fullscreen", True)

# Label
label = tk.Label(root, text="Digite uma palavra: ")
label.pack(padx=10, pady=10)

# texto
text = AutocompleteText(root, autocomplete=get_matches)
text.pack(padx=10, pady=10)

# Outras opções
texto_outras_opcoes = tk.Label(root, text="Outras opções: ")
texto_outras_opcoes.pack(padx=10, pady=10)

root.mainloop()